# How Nasty is Nero Really?

This notebook feeds the Nero data through the trained model and classifies it as either historical or mythological text

In [51]:
import os
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import collections
from collections import Counter

import plotly.express as px

print(tf.version.VERSION)

2.8.0


In [12]:
checkpoint_dir = "./ckpt"

In [7]:
def get_predicted_label_from_predictions(predictions):
    predicted_label = max([(predictions[l], l) for l in range(len(predictions))])[1]
    return predicted_label


In [23]:
def restore_model():
    # restore model from checkpoint
    checkpoints = [checkpoint_dir + "/" + name for name in os.listdir(checkpoint_dir)]
    
    if'./ckpt/.ipynb_checkpoints' in checkpoints:
        checkpoints.remove('./ckpt/.ipynb_checkpoints')
    
    latest_checkpoint = max(checkpoints, key=os.path.getctime)
    print(latest_checkpoint)
    print("Restoring from", latest_checkpoint)
    return keras.models.load_model(latest_checkpoint)


In [24]:
def make_array(dataframe):
    noNones = dataframe.applymap(lambda x:-1 if x == 'None' else x)
    toFloats = noNones.to_numpy().astype(float)
    return toFloats

In [38]:
def predictSection(file, model):
    df = pd.read_csv(file).drop(['Unnamed: 0'], axis=1)
    df = df.fillna(0)
    
    test_featues = make_array(df[[str(i) for i in range(7680)]])
    
    predictions = model.predict(x=test_featues)
    
    return [get_predicted_label_from_predictions(predictions[i]) for i,p in enumerate(predictions)]

In [41]:
# get trained model
model = restore_model()

['./ckpt/ckpt-loss=0.04']
./ckpt/ckpt-loss=0.04
Restoring from ./ckpt/ckpt-loss=0.04


In [46]:
# predict labels for nero dataset
predictions = []
for i in range(3):
    predictions += predictSection("nero-dfs/" + str(i) + ".csv", model)

In [47]:
# 1 is historical, 0 is mythological
Counter(predictions)

Counter({1: 1628, 0: 708})

In [48]:
# predict labels for Tacitus' Annals
annalsP = predictSection("nero-dfs/annals.csv", model)
Counter(annalsP)

Counter({1: 1174, 0: 399})

In [49]:
# predict labels for Suetonius' Lives of Caesars
bioP = predictSection("nero-dfs/biography.csv", model)
Counter(bioP)

Counter({1: 444, 0: 322})

In [57]:
# create data visualizations

nero = [1628, 708]
names = ['history', 'myth']
 
fig = px.pie(values=nero, names=names, title="Full Nero dataset")
fig.show()

In [54]:
nero = [1173, 399]
names = ['history', 'myth']
 
fig = px.pie(values=nero, names=names, title="Annals")
fig.show()

In [56]:
nero = [444, 322]
names = ['history', 'myth']
 
fig = px.pie(values=nero, names=names, title="Lives of Caesars")
fig.show()